### Mount ADLS containers for the Project

In [0]:
def mount_adls(storage_account_name, container_name):
    # Get id and secrets from key-vaults
    client_id = dbutils.secrets.get(scope='formula1-scope', key='formula1-databricks-client-id')
    tenant_id = dbutils.secrets.get(scope='formula1-scope', key='formula1-databricks-tenant-id')
    client_secret = dbutils.secrets.get(scope='formula1-scope', key='formula1-databricks-client-secret')

    # Set Spark Configuration
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
        }
    
    # Unmount the mount if it already available
    if any(mount.mountPoint == f'/mnt/{storage_account_name}/{container_name}' for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f'/mnt/{storage_account_name}/{container_name}')

    # Mount the storage container
    dbutils.fs.mount(
            source = f'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/',
            mount_point = f'/mnt/{storage_account_name}/{container_name}',
            extra_configs = configs
        )

    display(dbutils.fs.mounts())

##### Mount Raw Container

In [0]:
mount_adls('covid19reportingadfadls', 'raw')

##### Mount Processed Container

In [0]:
mount_adls('covid19reportingadfadls', 'processed')

##### Mount Lookup Container

In [0]:
mount_adls('covid19reportingadfadls', 'lookup')